In [ ]:
from app import app, db, bcrypt
from models import User, Stock, Portfolio
from flask import jsonify, request
import yfinance as yf
import pandas as pd
import sqlite3
import time

# Old lis
#companyNames = 'HDFCBANK.NS HINDUNILVR.NS ASIANPAINT.NS RELIANCE.NS TCS.NS AUROPHARMA.NS BBTC.NS BALKRISIND.NS GARFIBRES.NS COLPAL.NS NESTLEIND.NS MARUTI.NS BRITANNIA.NS BOSCHLTD.NS ONGC.NS ABBOTINDIA.NS DIVISLAB.NS BAJFINANCE.NS RECLTD.NS PFC.NS PIDILITIND.NS ARE&M.NS'
# Nifty 50 company symbols (Yahoo Finance format)

"""
companyNames = (
    'ADANIPORTS.NS APOLLOHOSP.NS ASIANPAINT.NS AXISBANK.NS BAJAJ-AUTO.NS BAJFINANCE.NS BAJAJFINSV.NS '
    'BPCL.NS BHARTIARTL.NS BRITANNIA.NS CIPLA.NS COALINDIA.NS DIVISLAB.NS DRREDDY.NS EICHERMOT.NS GRASIM.NS '
    'HCLTECH.NS HDFCBANK.NS HDFCLIFE.NS HEROMOTOCO.NS HINDALCO.NS HINDUNILVR.NS ICICIBANK.NS ITC.NS '
    'INDUSINDBK.NS INFY.NS JSWSTEEL.NS KOTAKBANK.NS LT.NS M&M.NS MARUTI.NS NESTLEIND.NS NTPC.NS '
    'ONGC.NS POWERGRID.NS RELIANCE.NS SBILIFE.NS SBIN.NS SUNPHARMA.NS TATAMOTORS.NS TATASTEEL.NS '
    'TCS.NS TECHM.NS TITAN.NS ULTRACEMCO.NS UPL.NS WIPRO.NS'
)
companyNamesArray = companyNames.split() 
multipleStocks = yf.Tickers(companyNames)
ms_stockprices = multipleStocks.history(period="max")

"""



companyNamesArray = [
    'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS',
    'BPCL.NS', 'BHARTIARTL.NS', 'BRITANNIA.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS',
    'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS',
    'ICICIBANK.NS', 'ITC.NS', 'INDUSINDBK.NS', 'INFY.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS',
    'MARUTI.NS', 'NESTLEIND.NS', 'NTPC.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SBIN.NS',
    'SUNPHARMA.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TCS.NS', 'TECHM.NS', 'TITAN.NS', 'ULTRACEMCO.NS', 'UPL.NS', 'WIPRO.NS'
]

all_data = []
batch_size = 10

for i in range(0, len(companyNamesArray), batch_size):
    batch = companyNamesArray[i:i+batch_size]
    tickers = " ".join(batch)
    try:
        ms = yf.Tickers(tickers)
        data = ms.history(period="max")
        all_data.append(data)
        print(f"Downloaded: {batch}")
    except Exception as e:
        print(f"Failed batch {batch}: {e}")
    time.sleep(10)  # Wait 10 seconds between batches


stockData_22 = all_data.loc[:,(['Close','Volume','Dividends','Stock Splits'],companyNamesArray)]
stockData_22.columns = stockData_22.columns.map(lambda x: f"{x[0]}_{x[1]}")
metrics = ['Close','Volume', 'Dividends', 'Stock Splits']

# Prepare the value_vars dictionary dynamically
value_vars = {}
for company in companyNamesArray:
    for metric in metrics:
        column_name = f"{metric}_{company}"
        if column_name in stockData_22.columns:  # Check if the column exists
            value_vars[column_name] = (company, metric)

df_long = stockData_22.melt(
    id_vars=["Date"], 
    var_name='variable',  # Use a scalar here
    value_name='value',
    value_vars=value_vars
)
# Split the 'variable' column into 'company' and 'metric'
df_long[['metric','company']] = df_long['variable'].str.split('_', expand=True)
df_long = df_long.drop(columns='variable')
new_order = ['Date','company','metric','value']
df_long=df_long[new_order]

# Create Metrics Table
metrics = df_long[['metric']].drop_duplicates().reset_index(drop=True)
metrics['Metric_ID'] = metrics.index + 1
metrics = metrics[['Metric_ID', 'metric']]

# Create Companies Table
companies = df_long[['company']].drop_duplicates().reset_index(drop=True)
companies['Company_ID'] = companies.index + 1
companies = companies[['Company_ID', 'company']]

# Create Data Table
data = df_long.merge(companies, on='company').merge(metrics, on='metric')
data = data[['Date', 'Company_ID', 'Metric_ID', 'value']]

# Connect to SQLite database (will create one if it doesn't exist)
conn = sqlite3.connect('stock_data_nifty50.db')

# Write the dataframes to SQL tables
metrics.to_sql('Metrics', conn, if_exists='replace', index=False)
companies.to_sql('Companies', conn, if_exists='replace', index=False)
data.to_sql('Data', conn, if_exists='replace', index=False)

# Close the connection
conn.close()



[*********************100%***********************]  4 of 5 completed

5 Failed downloads:
['APOLLOHOSP.NS', 'ASIANPAINT.NS', 'ADANIPORTS.NS', 'BAJAJ-AUTO.NS', 'AXISBANK.NS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Downloaded: ['ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS']


[*********************100%***********************]  5 of 5 completed

5 Failed downloads:
['BAJAJFINSV.NS', 'BRITANNIA.NS', 'BAJFINANCE.NS', 'BHARTIARTL.NS', 'BPCL.NS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Downloaded: ['BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BRITANNIA.NS']


[*********************100%***********************]  5 of 5 completed

5 Failed downloads:
['COALINDIA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS', 'EICHERMOT.NS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


...

[*********************100%***********************]  5 of 5 completed

5 Failed downloads:
['TITAN.NS', 'ULTRACEMCO.NS', 'TECHM.NS', 'TCS.NS', 'TATASTEEL.NS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Downloaded: ['TATASTEEL.NS', 'TCS.NS', 'TECHM.NS', 'TITAN.NS', 'ULTRACEMCO.NS']


[*********************100%***********************]  2 of 2 completed

2 Failed downloads:
['WIPRO.NS', 'UPL.NS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Downloaded: ['UPL.NS', 'WIPRO.NS']


AttributeError: 'list' object has no attribute 'loc'